<a href="https://colab.research.google.com/github/DeltaRCM/CSDMS_2022/blob/main/DeltaMetrics_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Part 2: Introduction to *DeltaMetrics*

([link](https://github.com/DeltaRCM/CSDMS_2022/blob/main/pyDeltaRCM_intro.ipynb) to part 1 of this workshop on the *pyDeltaRCM* numerical model)

<br>

Once we have data (whether from a numerical model, experiment, or remote sensing instrument) how do we analyze it? Are there published methods, metrics, and workflows which might be useful to us? Do we always know how to implement these methods?

<br>

For example: How would you want to quantify and examine the outputs of all of these different model results?

> <figure>
<img src="https://drive.google.com/uc?export=view&id=19D7O7MvXQMItCG-_yLVrlulrExkEwCiW" width="1000"/></figure>
Figure from Edmonds et al., 2022

__*DeltaMetrics* enables consistent and reproducible analysis of depositional systems, including deltas, by defining a standard data model and routines for metrics of system morphology and dynamics.__


## Aims of this notebook:

In this part of the workshop we plan to introduce you to a new analysis framework, _DeltaMetrics_, that we hope can reduce your "time to science" by providing easy access to published methods (letting you skip re-inventing the wheel and getting straight to your science). As we go through this notebook, we encourage you to think about how these types of methods might be applied to your models and data.

## What can one do with _DeltaMetrics_?

> <figure>
<img src="https://drive.google.com/uc?export=view&id=1bbPJC6IZ0hjnxRJz9DU4QwluNsM-wtiP" width="1000"/></figure>
Example animation of topography, corresponding binary land masks, and subaerial land area over time

<br>

_DeltaMetrics_ provides methods to deal with surface information, and its translation into the stratigraphy. 

A brief list of current functionality is provided below, a more comprehensive [user guide](https://deltarcm.org/DeltaMetrics/guides/userguide.html) is also available.

### Surface Methods

Surface methods include:

- Quick visualization of any data variables
- Planview operations
  - Shoreline roughness, length, avg. distance to
  - Channel widths and depths
  - Surface deposit times and ages
- Binary mask creation
  - Subaerial land extents
- Surface mobility quantification
  - 4 different published measures of surface channel mobility

### Subsurface Methods

Subsurface methods include:

- Creation of stratigraphy from timeseries of elevation data
- Visualization of stratigraphic slices
  - In-line with the rectilinear grid
  - Arbitrary slicing of the 3-D volume
  - Visualization of any data variable available

## Installation

First, we will install the package from GitHub; we plan to eventually release the package on PyPi.

In [ ]:
%%capture

!pip install git+https://github.com/DeltaRCM/DeltaMetrics.git
!pip install numba --upgrade

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import deltametrics as dm

## Introduction to the data model and tools in DeltaMetrics

*DeltaMetrics* centers around the use of “Cubes”. In *DeltaMetrics*, these `Cube` objects are the central office that connects all the different modules and a workflow together.
The base cube is the `DataCube`, which is set up to handle multi-variable three-dimensional datasets; for example, 2D-spatial timeseries data of multiple variables.
In fact, this data model is based on the *xarray* and *netCDF* data models, and *DeltaMetrics* is built on top of these packages ([more info on *xarray* data model](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#dataset)).

> <figure>
<img src="https://raw.githubusercontent.com/DeltaRCM/DeltaMetrics/develop/assets/private/3d_schema_cube.png" width="300"/></figure>
> Figure: Schematic representation of data in a `DataCube`. The first dimension is time (`t`), with the second and third being arbitrarily-named spatial dimensions (here, `x` and `y`).



The data of the `DataCube` can come from a file or can be directly passed; where possible, loading from a file is usually preferred, because it is memory-efficient.

Connecting to a netCDF file on disk is as simple as:

```
acube = dm.cube.DataCube('/path/to/data/file.nc')
```

For this guide to be easy to follow along with, we will use some sample *pyDeltaRCM* model output data that comes with *DeltaMetrics*.

In [ ]:
# load sample pyDeltaRCM model output data
golfcube = dm.sample_data.golf()
golfcube

In [ ]:
### this code should not be needed, except if Zenodo is not delivering data
#import urllib.request
#urllib.request.urlretrieve('https://utexas.box.com/shared/static/xypd02tjv3i5xyqzb9qtu8lpdip4vnpj.nc', 'golf.nc')
#golfcube = dm.cube.DataCube('golf.nc')
###

From our loaded `DataCube` we can query the "known" variables. In the generic sense, these variables are defined in the netCDF file that was loaded into the cube.

In [ ]:
golfcube.variables

Beyond the analytical capabilities of `DeltaMetrics`, visualization functionality is provided as well. Below we will very simply visualize the _final_ time instance for some of our known variables. Take a peek at the [`quick_show` documentation](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.cube.DataCube.html#deltametrics.cube.DataCube.quick_show) if you want to see the syntax for visualizing another time slice or wish to visualize the data along a different axis.

In [ ]:
golfcube.quick_show('eta')

In [ ]:
golfcube.quick_show('velocity')

The variables within the `DataCube` are more than standard NumPy arrays, they are [`xarray` DataArrays](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray). By using `xarray` all `DeltaMetrics` functionality keeps (or will keep) track of the _units_ associated with the different dimensions of the data. For example, our sample data variables have units of meters in the X and Y dimensions, and units of seconds in the temporal dimension.

In [ ]:
golfcube['eta']

Data returned when slicing the cube are `xarray` `DataArray` objects, so support arbitrary math.



In [ ]:
# compute the change in bed elevation between the last two intervals
diff_time = golfcube['eta'][-1, :, :] - golfcube['eta'][-2, :, :]
max_delta = abs(diff_time).max()

# make the plot
fig, ax = plt.subplots(figsize=(5, 3))
im = ax.imshow(
    diff_time, cmap='RdBu',
    vmax=max_delta,
    vmin=-max_delta)
cb = dm.plot.append_colorbar(im, ax)  # a convenience function
plt.show()


### Masks

One key element of many of the computations in DeltaMetrics is the use of `Mask` objects.

`Mask` objects are designed to identify morphological features (e.g., the shoreline), as well as geometric regions (e.g., a semi-circular area with some radius), to support further analyses based on the shape of the mask itself, or data properties within the masked region. ([List of all currently available Masks](https://deltarcm.org/DeltaMetrics/reference/mask/index.html))

<br>

In `DeltaMetrics` more generally, we aim to provide ready-to-use methods already established in the literature. For example, we provide the Opening Angle Method ([Shaw et al 2008](https://doi.org/10.1029/2008GL033963)) as well as a morphological method (based on [Geleynse et al 2012](https://doi.org/10.1029/2012GL052845)), to define delta shorelines. 


In [ ]:
# Creating a shoreline uses the Shaw et al., Opening Angle Method, by default

shorelinemask = dm.mask.ShorelineMask(
    golfcube['eta'][-1, :, :],
    elevation_threshold=golfcube.meta['H_SL'][-1],
    elevation_offset=0
)

The workflow of the Shaw et al., 2008 opening angle method is demonstrated in the documentation [here](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.plan.OpeningAnglePlanform.html#deltametrics.plan.OpeningAnglePlanform).

In [ ]:
# plot the shoreline overlaid on the final topography

# make a colormap for elevation data
cmap, norm = dm.plot.cartographic_colormap(
    H_SL=golfcube.meta['H_SL'][-1], h=4.5, n=1.0)

# make the plot
plt.imshow(golfcube['eta'][-1, :, :], cmap=cmap, norm=norm)
plt.imshow(shorelinemask.mask, alpha=0.4)
plt.show()

### Stratigraphy

If the loaded data includes a timeseries of elevation data, then we can compute the preserved information of data in the deposit as "stratigraphy".

DeltaMetrics has a special class of `Cube` for this, the `StratigraphyCube`

> <figure>
<img src="https://raw.githubusercontent.com/DeltaRCM/DeltaMetrics/develop/assets/private/3d_schema_stratcube.png" width="300"/></figure>
> Figure: Schematic representation of data in a `StratigraphyCube`. The first dimension is elevation (`z`), with the second and third being arbitrarily-named spatial dimensions (here, `x` and `y`).

[Full documentation on the `StratigraphyCube`](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.cube.StratigraphyCube.html#deltametrics.cube.StratigraphyCube)
Note: data in `StratigraphyCube` is created "on the fly", so can be computationally demanding. Learn more [here](https://deltarcm.org/DeltaMetrics/guides/examples/computations/comparing_speeds_of_stratigraphy_access.html) and how to `export` a frozen data set [here](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.cube.BaseCube.html#deltametrics.cube.BaseCube.export_frozen_variable).

<br>

#### Calculation for preservation and stratigraphy

Stratigraphy is calculated according to the preservation or erosion of periods in time, by identifying the last time a given elevation was occupied by the sediment surface.  

In [ ]:
# how is preservation calculated?
elevation_timeseries_example = np.array([0, 1, 3, 4, 2, 3, 4, 5, 4, 3, 4, 6, 7, 8])

dm.plot.show_one_dimensional_trajectory_to_strata(
    elevation_timeseries_example,
    )


In [ ]:
golfstrat = dm.cube.StratigraphyCube.from_DataCube(
    golfcube, dz=0.05
)

golfstrat

_Note_: We explicitly define the vertical resolution at which we grid this stratigraphy using the `dz` keyword argument above (there are other ways to [specify stratigraphic discretization](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.cube.StratigraphyCube.html#deltametrics.cube.StratigraphyCube.from_DataCube)).

In [ ]:
golfstrat.shape

In [ ]:
golfstrat['eta']

We can utilize the `quick_show()` method to visualize slices of the stratigraphy along any axis (x, y, or z), and to view the preserved quantities of any known variable (`golfstrat.variables`), try changing `eta` below to something like `velocity` or `stage` to see for yourself.

In [ ]:
golfstrat.variables

In [ ]:
golfstrat.quick_show('time', idx=70, colorbar_label=True)

DeltaMetrics also has routines to be able to slice `Cube` objects.
Here, we demonstrate a few different slice operations.

A simple example to begin with is slicing along the "strike" direction (or perpendicular to the flow of the inlet channel). 

> <figure>
<img src="https://raw.githubusercontent.com/DeltaRCM/DeltaMetrics/develop/assets/private/3d_schema_stratcubesection.png" width="300"/></figure>
> Figure: Schematic representation of slicing data from a `StratigraphyCube`. Here, a `strikeSection` is created across the deposit.

Slicing of arbitrary sections (i.e., not along x, y, or z axes) is supported by first defining a `Section` object.

In [ ]:
# set up three sections and show their paths
strike_section = dm.section.StrikeSection(
    golfstrat,
    distance=1000)
circular_section = dm.section.CircularSection(
    golfstrat,
    radius=2000)
arbitrary_section = dm.section.PathSection(
    golfstrat,
    path=np.array([[100, 3000],
                   [2000, 4000],
                   [2500, 7000]]))

fig, ax = plt.subplots()
golfcube.quick_show('eta', idx=-1)
strike_section.show_trace('r-', ax=ax)
circular_section.show_trace('g-', ax=ax)
arbitrary_section.show_trace('b-', ax=ax)

In [ ]:
# now view data from each of the sections that we created above
fig, ax = plt.subplots(3, 1, figsize=(8, 5))
strike_section.show('time', ax=ax[0])
circular_section.show('time', ax=ax[1])
arbitrary_section.show('time', ax=ax[2])
plt.tight_layout()

## Analyzing the effect of cross-basin slope

For this clinic, we precomputed the model runs to save us some time. We have integrated the model results as "sample data" in the DeltaMetrics package, so that you can easily load the results.

In [ ]:
xslope0, xslope1 = dm.sample_data.xslope()

In [ ]:
### this code should not be needed, except if Zenodo is not delivering data
#urllib.request.urlretrieve('https://utexas.box.com/shared/static/bc18byoa6ji7vq7jjjrb68dsxxtzavu4.nc', 'xslope0.nc')
#urllib.request.urlretrieve('https://utexas.box.com/shared/static/et6eng3e5bshgkls9h2lhg1tlactfaff.nc', 'xslope1.nc')
#xslope0, xslope1 = dm.cube.DataCube('xslope0.nc'), dm.cube.DataCube('xslope1.nc')
###

In [ ]:
nt = 5
ts = np.linspace(0, xslope0['eta'].shape[0]-1, num=nt, dtype=int)

fig, ax = plt.subplots(2, nt, figsize=(10, 2))
for i, t in enumerate(ts):
    ax[0, i].imshow(xslope0['eta'][t, :, :], vmin=-7, vmax=0.5)
    ax[0, i].set_title('t = ' + str(t))
    ax[1, i].imshow(xslope1['eta'][t, :, :], vmin=-7, vmax=0.5)

ax[1, 0].set_ylabel('dim1 direction')
ax[1, 0].set_xlabel('dim2 direction')

for axi in ax.ravel():
    axi.axes.get_xaxis().set_ticks([])
    axi.axes.get_yaxis().set_ticks([])

plt.show()

In [ ]:
# query the shape (t-x-y) of the cross-slope model
xslope0.shape

We can check the basin cross-slope variable defined in the cross-slope model and saved into the model's output file.

In [ ]:
xslope1.meta['basin_xslope']

### average shoreline distance

[Documentation for `compute_shoreline_distance`](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.plan.compute_shoreline_distance.html#deltametrics.plan.compute_shoreline_distance)

In [ ]:
fig, ax = plt.subplots(1, 2)
xslope0.quick_show('eta', idx=-1, ticks=True, ax=ax[0])
xslope1.quick_show('eta', idx=-1, ticks=True, ax=ax[1])
plt.tight_layout()
plt.show()

In [ ]:
xslope0_shorelinemask = dm.mask.ShorelineMask(
    xslope0['eta'][-1, :, :],
    elevation_threshold=0.0
)

xslope1_shorelinemask = dm.mask.ShorelineMask(
    xslope1['eta'][-1, :, :],
    elevation_threshold=0.0
)

In [ ]:
xslope0_shore_mean, xslope0_shore_std = dm.plan.compute_shoreline_distance(
    xslope0_shorelinemask, origin=[xslope0.meta['CTR'].data, xslope0.meta['L0'].data])

xslope1_shore_mean, xslope1_shore_std = dm.plan.compute_shoreline_distance(
    xslope1_shorelinemask, origin=[xslope1.meta['CTR'].data, xslope1.meta['L0'].data])

In [ ]:
print(
    f'xslope0 shoreline distance: {xslope0_shore_mean:0.2f} +/- {xslope0_shore_std:0.2f}'
)

print(
    f'xslope1 shoreline distance: {xslope1_shore_mean:0.2f} +/- {xslope1_shore_std:0.2f}'
)


Instead of measuring the distance to the _entire_ shoreline, we can apply a geometric mask to only examine the downslope side of the basin. 

In [ ]:
# do geometric masking to compare only on the downslope side?
downslope_mask = dm.mask.GeometricMask(xslope0['eta'][-1, :, :])
downslope_mask.angular(np.pi/10, 3*np.pi/7)
downslope_mask.circular(20, 100)
downslope_mask.show()

There are a number of other binary masks built into _DeltaMetrics_, a partial list is provided below, a full list and example functionality is provided in the [documentation](https://deltarcm.org/DeltaMetrics/reference/mask/index.html).

- Binary masks
  - Elevation-based
  - Flow field-based
  - Subaerial extents
  - Shoreline delineation
  - Channel locations
  - Wet pixels
  - Land-water edges
  - Channel centerlines
  - Deposit extents
  - Based on thresholds
  - Geometric shapes
    - Rectangular
    - Radial
    - Circular

We can visualize the delta elevation and identified shoreline across the full system, and after we apply our radial mask to isolate just the downslope region.

In [ ]:
# make a colormap for elevation data
cmap, norm = dm.plot.cartographic_colormap(
    H_SL=xslope0.meta['H_SL'][-1], h=4.5, n=1.0)

In [ ]:
fig, ax = plt.subplots(2, 2)
for i in range(2):
  # plot the topographies
  if i == 0:
    ax[i, 0].imshow(xslope0['eta'][-1, :, :], cmap=cmap, norm=norm)
    ax[i, 1].imshow(xslope1['eta'][-1, :, :], cmap=cmap, norm=norm)
  else:
    ax[i, 0].imshow(xslope0['eta'][-1, :, :]*downslope_mask.mask, cmap=cmap, norm=norm)
    ax[i, 1].imshow(xslope1['eta'][-1, :, :]*downslope_mask.mask, cmap=cmap, norm=norm)   

  # show the shorelines on top
  if i == 0:
    ax[i, 0].imshow(xslope0_shorelinemask.mask, alpha=0.4)
    ax[i, 1].imshow(xslope1_shorelinemask.mask, alpha=0.4)
  else:
    ax[i, 0].imshow(xslope0_shorelinemask.mask*downslope_mask.mask, alpha=0.4)
    ax[i, 1].imshow(xslope1_shorelinemask.mask*downslope_mask.mask, alpha=0.4)

plt.tight_layout()
plt.show()

We can repeat our shoreline distance calculation by passing masked versions of the shoreline array to the `compute_shoreline_distance()` function. We can also ask the function to return the full set of distance values to each location along the shoreline, in addition to the mean and standard deviation.

In [ ]:
ds_xslope0_shore_mean, ds_xslope0_shore_std, ds_xslope0_distances = dm.plan.compute_shoreline_distance(
    xslope0_shorelinemask.mask*downslope_mask.mask,
    origin=[xslope0.meta['CTR'].data, xslope0.meta['L0'].data],
    return_distances=True)

ds_xslope1_shore_mean, ds_xslope1_shore_std, ds_xslope1_distances = dm.plan.compute_shoreline_distance(
    xslope1_shorelinemask.mask*downslope_mask.mask,
    origin=[xslope1.meta['CTR'].data, xslope1.meta['L0'].data],
    return_distances=True)

In [ ]:
print(
    f'xslope0 shoreline distance: {ds_xslope0_shore_mean:0.2f} +/- {ds_xslope0_shore_std:0.2f}'
)

print(
    f'xslope1 shoreline distance: {ds_xslope1_shore_mean:0.2f} +/- {ds_xslope1_shore_std:0.2f}'
)


In [ ]:
# plot box plots of distance values to individual points along the shoreline
plt.boxplot(ds_xslope0_distances, positions=[0])
plt.boxplot(ds_xslope1_distances, positions=[1])
plt.show()

In [ ]:
# are the distance distributions statistically different?
import scipy.stats as stats

stats.ttest_ind(ds_xslope0_distances, ds_xslope1_distances)

### Stratigraphic difference

In the interest of time, we'll make a quick plot of a strike section (perpendicular to the direction of the inlet channel) to see the impact of the sloped basin on the resulting deposit.

In [ ]:
# first we define our stratigraphy cubes from the data cubes
xslope0strat = dm.cube.StratigraphyCube.from_DataCube(xslope0, dz=0.05)
xslope1strat = dm.cube.StratigraphyCube.from_DataCube(xslope1, dz=0.05)

In [ ]:
# then we can quickly visualize them along a specified index and axis
fig, ax = plt.subplots(2, 1)
xslope0strat.quick_show('time', idx=20, axis=1, ax=ax[0])
xslope1strat.quick_show('time', idx=20, axis=1, ax=ax[1])

for axi in ax.ravel():
  axi.set_ylim(-10, 1)

## Testing your own ideas with DeltaMetrics

At this point in the clinic, we want you to explore your own ideas for measuring attributes of the deltas and deposits.

You can [check out the documentation here](https://deltarcm.org/DeltaMetrics/) to see what we've already implemented.

Some suggestions:
* how does the location of channels change over time? (see [`mobility`](https://deltarcm.org/DeltaMetrics/reference/mobility/index.html))
* what is the geometry of channels moving downstream? (see [`compute_channel_width`](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.plan.compute_channel_width.html#deltametrics.plan.compute_channel_width) and [`compute_channel_depth`](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.plan.compute_channel_depth.html#deltametrics.plan.compute_channel_depth))
* what is the age of the deposit surface? (see [`compute_surface_deposit_age`](https://deltarcm.org/DeltaMetrics/_autosummary/deltametrics.plan.compute_surface_deposit_age.html#deltametrics.plan.compute_surface_deposit_age))


Hint: try to develop your metric on just one cube at a time, or even just one timestep of one cube. Then, we can wrap it up to make it work for the whole timeseries!


### End + Call to Action

This is the end of the workshop, thank you for joining us! The `DeltaMetrics` package itself is under active development and we welcome any users and code contributors to join the team.

*Have some model output, experimental data, or remote sensing observations you'd like to analyze using the `DeltaMetrics` package but don't know where to start or how to format your data?*

Reach out! We'd be happy to help you get things set up and connected to the `DeltaMetrics` ecosystem. Would you rather try it out your own? Check out this [guide for data formatting](https://deltarcm.org/DeltaMetrics/guides/examples/io/setup_any_dataset.html) or a [quick way to just check out the tools on your own data](https://deltarcm.org/DeltaMetrics/guides/examples/io/connect_to_nonstandard_data.html).

<figure>
<img src="https://deltarcm.org/DeltaMetrics/guides/examples/computations/aggradation_preserved_time-2.hires.png" width="500"/></figure>

Figure: DeltaMetrics works with any two-dimensional depositional system model results. Here, results from the Swanson et al., 2017 numerical dune field model are used to assess the effect of aggradation on time preservation in strata. See the [`aeolian` data here](https://deltarcm.org/DeltaMetrics/reference/sample_data/index.html#deltametrics.sample_data.aeolian) and [the example here](https://deltarcm.org/DeltaMetrics/guides/examples/computations/aggradation_preserved_time.html).

<br>
<figure>
<img src="https://deltarcm.org/DeltaMetrics/reference/sample_data/index-4.hires.png" width="900"/></figure>

Figure: DeltaMetrics works with time series satellite imagery. Here, a set of satellite images from the Landsat 5 satellite, collected over the Krishna River delta, India. The dataset includes annual-composite scenes from four different years ([1995, 2000, 2005, 2010]) and includes data collected from four bands ([‘Red’, ‘Green’, ‘Blue’, ‘NIR’]).. See the [`landsat` data here](https://deltarcm.org/DeltaMetrics/reference/sample_data/index.html#deltametrics.sample_data.landsat).